<a href="https://colab.research.google.com/github/HoseinNekouei/PyTorch_Binary-Prediction-of-Poisonous-Mushrooms/blob/main/Torch_binary_prediction_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Dataset**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir /content/data

mkdir: cannot create directory ‘/content/data’: File exists


In [3]:
!cp /content/drive/MyDrive/dataset/playground-series-s4e8_2.zip /content/data

In [4]:
!unzip /content/data/playground-series-s4e8_2.zip -d /content/data -A

Archive:  /content/data/playground-series-s4e8_2.zip
replace /content/data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/data/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/data/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [5]:
# !rm /content/test.csv
# !rm /content/train.csv
# !rm /content/sample_submission.csv

# **Import Library**

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


# **Preprocessing**

  # *Load Train set*

In [8]:
train_df= pd.read_csv('/content/data/train.csv')
train_df

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


# *NaN*

In [10]:
train_df= train_df.drop(columns= ['id'])
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   class                 object 
 1   cap-diameter          float64
 2   cap-shape             object 
 3   cap-surface           object 
 4   cap-color             object 
 5   does-bruise-or-bleed  object 
 6   gill-attachment       object 
 7   gill-spacing          object 
 8   gill-color            object 
 9   stem-height           float64
 10  stem-width            float64
 11  stem-root             object 
 12  stem-surface          object 
 13  stem-color            object 
 14  veil-type             object 
 15  veil-color            object 
 16  has-ring              object 
 17  ring-type             object 
 18  spore-print-color     object 
 19  habitat               object 
 20  season                object 
dtypes: float64(3), object(18)
memory usage: 499.4+ MB


In [11]:
train_df.isna().mean()

,0
class,0.000000
cap-diameter,0.000001
cap-shape,0.000013
cap-surface,0.215282
cap-color,0.000004
does-bruise-or-bleed,0.000003
gill-attachment,0.168093
gill-spacing,0.403740
gill-color,0.000018
stem-height,0.000000


In [12]:
missing_treshold= 0.85

high_missing_columns= train_df.columns[train_df.isna().mean() > missing_treshold]
high_missing_columns

train_df= train_df.drop(columns= high_missing_columns)
train_df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season
0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,w,f,f,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,y,o,t,z,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,s,n,f,f,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,w,f,f,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,w,f,f,g,a


In [13]:
mode_values= train_df.mode()
mode_values= mode_values.loc[0].to_dict()
mode_values

{'class': 'p',
 'cap-diameter': 1.49,
 'cap-shape': 'x',
 'cap-surface': 't',
 'cap-color': 'n',
 'does-bruise-or-bleed': 'f',
 'gill-attachment': 'a',
 'gill-spacing': 'c',
 'gill-color': 'w',
 'stem-height': 5.24,
 'stem-width': 2.41,
 'stem-surface': 's',
 'stem-color': 'w',
 'has-ring': 'f',
 'ring-type': 'f',
 'habitat': 'd',
 'season': 'a'}

In [14]:
median_values= train_df.median(numeric_only= True).to_dict()
median_values

{'cap-diameter': 5.75, 'stem-height': 5.88, 'stem-width': 9.65}

In [15]:
for column in train_df:
  if train_df[column].isna().any():
    if train_df[column].dtype== 'object':
      train_df[column]= train_df[column].fillna(mode_values[column])
    else:
      train_df[column]= train_df[column].fillna(median_values[column])

train_df.isna().median()

,0
class,0.0
cap-diameter,0.0
cap-shape,0.0
cap-surface,0.0
cap-color,0.0
does-bruise-or-bleed,0.0
gill-attachment,0.0
gill-spacing,0.0
gill-color,0.0
stem-height,0.0


In [16]:
train_df[train_df.isna().any(axis=1)]

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season


# *Split Dataframe to train and validation*

In [17]:
num_samples, num_features= train_df.shape
num_features-= 1
num_classes= len(train_df['class'].unique())
print(f'num_sample: {num_samples}, num_features: {num_features}, num_classes: {num_classes}')

num_sample: 3116945, num_features: 16, num_classes: 2


In [18]:
train_set, val_set= train_test_split(train_df, test_size=0.2, random_state=42)
print(f'train_set: {train_set.shape}, val_set: {val_set.shape}')

train_set: (2493556, 17), val_set: (623389, 17)


# *Encoding*

In [19]:
ordinal_encoder= OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value= -1)
cat_col_train= train_set.select_dtypes(include=['object']).columns
print(cat_col_train)

train_set[cat_col_train]= ordinal_encoder.fit_transform(train_set[cat_col_train]).astype(str)
val_set[cat_col_train]= ordinal_encoder.transform(val_set[cat_col_train]).astype(str)

train_set.head(2)

Index(['class', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-surface', 'stem-color', 'has-ring', 'ring-type', 'habitat',
       'season'],
      dtype='object')


(        class  cap-diameter cap-shape cap-surface cap-color  \
 1252551   1.0          2.58      58.0        70.0      52.0   
 1799166   1.0          1.83      41.0        46.0      53.0   
 
         does-bruise-or-bleed gill-attachment gill-spacing gill-color  \
 1252551                 19.0            40.0         24.0       41.0   
 1799166                  8.0            37.0         24.0       41.0   
 
          stem-height  stem-width stem-surface stem-color has-ring ring-type  \
 1252551         2.87        5.69         44.0       41.0      4.0      16.0   
 1799166         5.36        2.70         44.0       41.0      4.0      16.0   
 
         habitat season  
 1252551    23.0    0.0  
 1799166    34.0    0.0  ,
         class  cap-diameter cap-shape cap-surface cap-color  \
 1356763   1.0          7.82      58.0        67.0      55.0   
 2369311   1.0          3.20      55.0        67.0      46.0   
 
         does-bruise-or-bleed gill-attachment gill-spacing gill-color 

In [20]:
# Delete unused variable
del train_df

# Force  garbage collection
import gc
gc.collect()

120

In [21]:
# Check current RAM usage
import psutil
ram_usage = psutil.virtual_memory()
print(f"Used RAM: {ram_usage.used / (1024 ** 3):.2f} GB")
print(f"Available RAM: {ram_usage.available / (1024 ** 3):.2f} GB")

Used RAM: 6.23 GB
Available RAM: 6.16 GB


# *Normalization*

In [22]:
num_col_train= train_set.select_dtypes(include= 'float64').columns

scaler= StandardScaler()

train_set[num_col_train]= scaler.fit_transform(train_set[num_col_train])
val_set[num_col_train]= scaler.transform(val_set[num_col_train])

train_set.head(2)

,cap-diameter,stem-height,stem-width
1252551,-0.800811,-1.288253,-0.674931
1799166,-0.961857,-0.366103,-1.044348
1936146,-0.233929,0.359766,-0.462424
1464811,-0.384239,-0.184636,0.196103
767639,-0.027790,-0.006872,-0.106597
